# I. Loading/Plot functions

In [3]:
DATASETS = {
    'a': 'dogs',
    'b': 'medical-leaf',
    'c': 'texture-dtd',
    'd': 'birds',
    'e': 'AWA',
    'f': 'plt-net',
    'g': 'resisc',
    'h': 'plt-doc',
    'i': 'airplanes',
    '_': 'ALL'
}

color_dict = {
        'erm': 'blue',
        'jtt': 'red',
        'suby': '#00CC96',  # green
        'subg': '#ff7f0e',  # orange
        'rwy': '#00CC96',
        'rwg': '#ff7f0e',
        'dro': '#DEA0FD'  # purple
    }
    

In [4]:
import wandb
import pandas as pd
import plotly.graph_objects as go
import numpy as np

# Initialize wandb
wandb.login()

# Set your entity and project
entity_name = "aureliengauffre"  # e.g., your username or team name
project_name = "SMA_all_2_best"

# Initialize the wandb API ||client
api = wandb.Api()

# Fetch all runs from the specified project
runs = api.runs(f"{entity_name}/{project_name}")

# Create an empty list to hold data for each run
data = []

# Loop through runs and ext/ract the data you're interested in
for run in runs:
    # Extract both summary metrics and config (hyperparameters) for each run
    run_data = {
        "name": run.name,
        "summary_metrics": run.summary._json_dict,
        "config": run.config,
        # Add any other attributes you're interested in here
    }
    data.append(run_data)

# Convert the list of data to a pandas DataFrame
df = pd.DataFrame(data)

# For summary metrics and config (hyperparameters), expand them into separate columns
df_summary = pd.json_normalize(df['summary_metrics'])
df_config = pd.json_normalize(df['config'])
df = pd.concat([df.drop(['summary_metrics', 'config'], axis=1), df_summary, df_config], axis=1)
df = df.iloc[:, 1:]  # Drop the first column containing the name of the runs (since there is also another name column)

# Now we have a DataFrame `df` with all runs, their summary metrics, and hyperparameters
# print(df.head())  # Print the first few rows of the DataFrame


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aureliengauffre (use `wandb login --relogin` to force relogin)


In [5]:
df['name']

0            plt-doc
1            plt-doc
2            plt-net
3            plt-net
4            plt-net
            ...     
3561       airplanes
3562       airplanes
3563    medical-leaf
3564       airplanes
3565       airplanes
Name: name, Length: 3566, dtype: object

In [6]:
# Initialize a Plotly figure
def plot_graph_old(df, x_axis, y_axis, dataset_name, methods=None):
    # Default methods if not provided
    
    if methods is None:
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    
    if dataset_name is not None:
        df_dataset = df[(df['name'] == dataset_name) ]#& (df['mu'] >= .1) ]
    else :
        df_dataset = df 
    fig = go.Figure() # Initialize a Plotly figure
    for method in methods:
        df_method = df_dataset[df_dataset['method'] == method]
        # Group by x_axis and calculate the mean of y_axis
        df_avg = df_method.groupby(x_axis)[y_axis].mean().reset_index()
        
        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line to the plot for the current method
        color = color_dict.get(method, 'grey')
        fig.add_trace(go.Scatter(x=df_avg[x_axis], y=df_avg[y_axis], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color)))
    
    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis} , dataset={dataset_name}',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=600   # Height of the figure in pixels
                     )

    # Show the figure
    fig.show()

# Example usage (assuming df is your DataFrame):
# plot_graph(df, 'K', 'mean_gr


dataset_name = 'AWA' #73sports
y_axis = 'mean_grp_acc_te' #'mean_grp_acc_te'
x_axis = 'K'
plot_graph(df, x_axis, y_axis, dataset_name, methods=None)


In [62]:
import plotly.graph_objects as go


def normalize_df(df,metric = 'best_acc_te'):
    # Copy the DataFrame to avoid modifying the original data
    result_df = df.copy()
    
    # Group by 'name', 'K', 'mu', and 'init_seed' and calculate the minimum and maximum accuracy
    grouped = df.groupby(['name', 'K', 'mu', 'init_seed'])[metric]
    min_acc = grouped.transform(np.min)
    max_acc = grouped.transform(np.max)
    
    # Apply the Min-Max normalization formula
    result_df[metric] = (df[metric] - min_acc) / (max_acc - min_acc)
    
    return result_df




def plot_graph(df, x_axis, y_axis, dataset_name, error_bars=None, normalize = False, methods=None,):
    print(dataset_name)
    if methods is None or methods == 'ALL':
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    color_dict = {
            'erm': 'blue',
            'jtt': 'red',
            'suby': '#00CC96',  # green
            'subg': '#ff7f0e',  # orange
            'rwy': '#00CC96',
            'rwg': '#ff7f0e',
            'dro': '#DEA0FD'  # purple
        }
    if dataset_name is None or dataset_name == 'ALL': 
        df_dataset = df
    else:
        df_dataset = df[(df['name'] == dataset_name)]
    
    if normalize :
        df_dataset = normalize_df(df_dataset,metric=y_axis)
        
    fig = go.Figure()  # Initialize a Plotly figure
    for method in methods:
        df_method = df_dataset[df_dataset['method'] == method]
        # Group by x_axis and calculate the mean, standard deviation, and count (for standard error calculation)
        stats = df_method.groupby(x_axis)[y_axis].agg(['mean', 'std', 'count']).reset_index()
        
        # Calculate standard error (SEM)
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])
        #print(method, stats) # print the number on which we average, interesting !
        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line with error bars to the plot for the current method
        color = color_dict.get(method, 'grey')
        if error_bars :
            fig.add_trace(go.Scatter(x=stats[x_axis], y=stats['mean'], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color),
                                 error_y=dict(type='data', array=stats['sem'], visible=True)))
        else :
            fig.add_trace(go.Scatter(x=stats[x_axis], y=stats['mean'], mode='lines+markers',
                                 name=method, line=dict(dash=line_style, color=color)))
    
    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis}, dataset={dataset_name}',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=800  # Height of the figure in pixels
                     )

    # Show the figure
    
    fig.show()


def plot_graph_all(df, x_axis, y_axis, error_bars=None, normalize=False, methods=None):
    """Unlike the original plot_graph function which calculates and plots error bars
    based on individual datasets directly, this version computes the standard error within each dataset first 
    and then averages these errors across all datasets for each method. This approach provides a generalized 
    view of method performance and variability across different datasets."""
    
    if methods is None or methods == 'ALL':
        methods = ['erm', 'jtt', 'suby', 'subg', 'rwy', 'rwg', 'dro']
    dashed_methods = ['rwy', 'rwg', 'dro']
    
    if normalize:
        df = normalize_df(df, metric=y_axis)

    fig = go.Figure()  # Initialize a Plotly figure
    all_stats = pd.DataFrame()

    # Process each method separately
    for method in methods:
        df_method = df[df['method'] == method]

        # Group data first by 'name' and then by x_axis and compute statistics
        grouped = df_method.groupby(['name', x_axis])
        stats = grouped[y_axis].agg(['mean', 'std', 'count']).reset_index()

        # Calculate standard error within each dataset
        stats['sem'] = stats['std'] / np.sqrt(stats['count'])
        
        # Append results to the all_stats DataFrame for later display
        stats['method'] = method
        all_stats = pd.concat([all_stats, stats], ignore_index=True)
        
        # Now group by x_axis and calculate the mean of the means and the mean of the SEMs
        final_stats = stats.groupby(x_axis).agg({'mean': 'mean', 'sem': 'mean'}).reset_index()

        # Determine line style based on whether method is in dashed_methods
        line_style = 'dash' if method in dashed_methods else 'solid'
        
        # Add a line with error bars to the plot for the current method
        color = color_dict.get(method, 'grey')
        if error_bars:
            fig.add_trace(go.Scatter(x=final_stats[x_axis], y=final_stats['mean'], mode='lines+markers',
                                     name=method, line=dict(dash=line_style, color=color),
                                     error_y=dict(type='data', array=final_stats['sem'], visible=True)))
        else:
            fig.add_trace(go.Scatter(x=final_stats[x_axis], y=final_stats['mean'], mode='lines+markers',
                                     name=method, line=dict(dash=line_style, color=color)))

    # Update the layout
    fig.update_layout(title=f'{y_axis} vs {x_axis}, Average on all dataset (mean and std)',
                      xaxis_title=x_axis,
                      yaxis_title=y_axis,
                      legend_title='Method',
                      width=800,  # Width of the figure in pixels
                      height=800  # Height of the figure in pixels
                     )

    # Show the figure
    fig.show()

    # Display the all_stats DataFrame
    return all_stats.pivot_table(index=[x_axis, 'name'], columns='method', values=['count', 'sem'], aggfunc='first')



# II. Analysis : K

Here are the **plot parameters** to be played with :

In [63]:
ERROR_BARS = True # Wether to plot the error bars
NORMALIZE = False

To analyse the impact of K, we the **value of mu is fixed**:

In [72]:
df_fix_mu = df[df['mu']==.05] # Currently mu in [.05,.1,.2,.4]

df_fix_mu_K2 = df[df['K']==2] 
df_fix_mu_K4 = df[df['K']==4] 
df_fix_mu_K8 = df[df['K']==8] 
df_fix_mu_K12 = df[df['K']==12] 

## a. best_acc_te

In [73]:
x_axis = 'K'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)


count                                        sem            \
method           dro  erm   jtt  rwg  rwy  subg  suby       dro       erm   
K name                                                                      
2 AWA            5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.004747  0.004054   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.026757  0.022423   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.009868  0.008473   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.035725  0.018371   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.016197  0.013920   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.030264  0.016465   
  plt-net        5.0  5.0  10.0  5.0  5.0  10.0  10.0  0.010854  0.001717   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.002964  0.013566   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.011744  0.019270   
4 AWA            5.0  5.0   5.0  5.0  5.0  10.0   5.0  0.003190  0.003885   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.009539  0.010725   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.001275  0.001531   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.011016  0.006940   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.011374  0.015285   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.003576  0.011876   
  plt-net        1.0  5.0   NaN  NaN  NaN   NaN   NaN       NaN  0.001587   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.020028  0.012332   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.011940  0.016236   
8 AWA            5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.002992  0.002323   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.010417  0.002784   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.000639  0.000648   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.003001  0.003588   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.003857  0.012974   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.002929  0.006227   
  plt-net        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.001933  0.001055   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.004824  0.005187   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.009783  0.006837   

                                                                  
method               jtt       rwg       rwy      subg      suby  
K name                                                            
2 AWA           0.007132  0.006483  0.010450  0.001766  0.005871  
  airplanes     0.031813  0.033598  0.013061  0.010423  0.010402  
  birds         0.015478  0.010897  0.009312  0.009032  0.017364  
  dogs          0.020373  0.018918  0.007196  0.017807  0.015543  
  medical-leaf  0.031995  0.009443  0.014640  0.036265  0.029514  
  plt-doc       0.024901  0.016794  0.007456  0.016770  0.010886  
  plt-net       0.006165  0.009055  0.012012  0.003042  0.004424  
  resisc        0.007819  0.017274  0.007656  0.022949  0.010064  
  texture-dtd   0.010440  0.012917  0.009160  0.045728  0.021916  
4 AWA           0.005705  0.005085  0.005435  0.004109  0.002976  
  airplanes     0.007702  0.010926  0.004434  0.009269  0.008506  
  birds         0.001989  0.001680  0.000884  0.000622  0.001915  
  dogs          0.008785  0.006708  0.005702  0.010057  0.016070  
  medical-leaf  0.009660  0.018760  0.008245  0.021000  0.027321  
  plt-doc       0.013196  0.002780  0.006386  0.010614  0.006044  
  plt-net            NaN       NaN       NaN       NaN       NaN  
  resisc        0.017692  0.005776  0.004534  0.003151  0.014347  
  texture-dtd   0.011970  0.007345  0.011122  0.011970  0.014827  
8 AWA           0.002195  0.003131  0.005197  0.001333  0.005126  
  airplanes     0.001780  0.002696  0.003501  0.004118  0.006740  
  birds         0.000754  0.001013  0.000500  0.000399  0.000303  
  dogs          0.003823  0.003697  0.006104  0.003655  0.006033  
  medical-leaf  0.024009  0.0

In [70]:
NORMALIZE

False

In [74]:
x_axis = 'K'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, normalize=NORMALIZE)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## b. worst_acc_te

In [75]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=False, normalize=NORMALIZE)

count                                        sem             \
method           dro  erm   jtt  rwg  rwy  subg  suby       dro        erm   
K name                                                                       
2 AWA            5.0  5.0   5.0  5.0  5.0   5.0   5.0  5.285351   7.815770   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  9.015927   4.307786   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  5.909254  14.278238   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  1.269381   2.218810   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  1.176471   1.176471   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  2.113973   0.827737   
  plt-net        5.0  5.0  10.0  5.0  5.0  10.0  10.0  2.684134   0.995766   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  1.452186   1.265270   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  8.888889   3.664141   
4 AWA            5.0  5.0   5.0  5.0  5.0  10.0   5.0  4.160248   3.901398   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  8.719997   4.673160   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.902846   0.628074   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  4.800716   1.688531   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  3.463401   6.430667   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.984982   0.863433   
  plt-net        2.0  5.0   NaN  NaN  NaN   NaN   NaN  0.709995   2.557905   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  4.678799   7.600274   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  2.576005   1.039349   
8 AWA            5.0  5.0   5.0  5.0  5.0   5.0   5.0  2.913004   2.827365   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.303030   0.255472   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.504067   0.500760   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.677408   0.596249   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  1.412508   1.474230   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.977615   2.181480   
  plt-net        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.323238   2.817479   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  5.531604   2.552843   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.000000   0.000000   

                                                                   
method               jtt        rwg       rwy      subg      suby  
K name                                                             
2 AWA           4.474268   8.141688  4.872016  1.386377  6.253393  
  airplanes     0.927458   6.662022  5.712818  4.630114  1.496619  
  birds         2.968569  10.256449  7.664581  2.671972  5.154132  
  dogs          1.538159   2.567449  1.059739  4.139578  1.644408  
  medical-leaf  1.440876   1.440876  2.352941  5.346400  4.822336  
  plt-doc       2.847008   5.062968  2.510024  2.963089  0.437409  
  plt-net       2.001805   2.996579  5.101449  1.226934  1.131341  
  resisc        1.385051   0.968904  2.220294  7.880341  2.270785  
  texture-dtd   3.685139   7.954345  6.047650  5.241101  2.421611  
4 AWA           6.215691   4.692563  3.960460  1.488967  4.094905  
  airplanes     3.497415   3.262211  5.015975  3.067890  5.313958  
  birds         1.730366   0.644938  0.467041  1.881298  1.870886  
  dogs          3.973173   1.656076  3.583545  5.916469  1.865240  
  medical-leaf  2.630668   4.962838  3.421142  2.200605  2.261243  
  plt-doc       1.866966   1.611635  1.107338  1.665806  0.976014  
  plt-net            NaN        NaN       NaN       NaN       NaN  
  resisc        2.158147   4.901293  4.970662  4.043415  5.855400  
  texture-dtd   1.842569   2.421611  2.041241  2.078699  1.619709  
8 AWA           2.594125   2.725117  4.078984  3.056242  1.713918  
  airplanes     0.220386   0.654234  0.186841  3.855175  0.140469  
  birds         0.655119   0.286355  0.829958  0.590586  0.545743  
  dogs          0.865555   1.040422  1.376

In [13]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## c. relative_acc

In [76]:
x_axis = 'K'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=False, normalize=NORMALIZE)

count                                        sem            \
method           dro  erm   jtt  rwg  rwy  subg  suby       dro       erm   
K name                                                                      
2 AWA            5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.023009  0.032909   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.053216  0.028260   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.023893  0.081425   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.016514  0.011094   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.019066  0.012334   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.027198  0.035198   
  plt-net        5.0  5.0  10.0  5.0  5.0  10.0  10.0  0.009724  0.005037   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.020655  0.035743   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.036957  0.024296   
4 AWA            5.0  5.0   5.0  5.0  5.0  10.0   5.0  0.007931  0.009687   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.014647  0.009982   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.001419  0.000502   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.014967  0.008363   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.013739  0.023143   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.004885  0.010445   
  plt-net        2.0  5.0   NaN  NaN  NaN   NaN   NaN  0.000298  0.005415   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.029091  0.014598   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.021041  0.019770   
8 AWA            5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.002695  0.002594   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.017405  0.003922   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.000663  0.000719   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.005146  0.004766   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.007831  0.013024   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.004361  0.008860   
  plt-net        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.001770  0.000710   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.012569  0.007232   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.006912  0.002041   

                                                                  
method               jtt       rwg       rwy      subg      suby  
K name                                                            
2 AWA           0.019411  0.038138  0.025972  0.012687  0.027103  
  airplanes     0.007513  0.077661  0.047595  0.084258  0.026313  
  birds         0.008835  0.048309  0.038195  0.017649  0.034727  
  dogs          0.017870  0.011521  0.014715  0.067297  0.015656  
  medical-leaf  0.044998  0.018169  0.028794  0.066516  0.044626  
  plt-doc       0.034602  0.062536  0.052096  0.108599  0.023398  
  plt-net       0.009984  0.012773  0.038393  0.018111  0.009785  
  resisc        0.091196  0.039923  0.014712  0.109069  0.019830  
  texture-dtd   0.019642  0.042898  0.026860  0.111077  0.029593  
4 AWA           0.006736  0.006937  0.011130  0.004982  0.005812  
  airplanes     0.009730  0.015785  0.004898  0.025768  0.021532  
  birds         0.000995  0.002066  0.001808  0.001889  0.002740  
  dogs          0.011766  0.014014  0.011482  0.015176  0.018321  
  medical-leaf  0.017323  0.021612  0.009164  0.042502  0.021645  
  plt-doc       0.021295  0.023430  0.009793  0.077359  0.015724  
  plt-net            NaN       NaN       NaN       NaN       NaN  
  resisc        0.006070  0.008286  0.009337  0.011096  0.021097  
  texture-dtd   0.014065  0.012562  0.021921  0.038153  0.027845  
8 AWA           0.001753  0.004182  0.006205  0.004864  0.006126  
  airplanes     0.001671  0.013651  0.012338  0.008108  0.007368  
  birds         0.000367  0.001182  0.000749  0.000927  0.000961  
  dogs          0.006730  0.003690  0.004276  0.006805  0.007479  
  medical-leaf  0.010243  0.0

In [14]:
x_axis = 'K'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## d. minor_group_acc

In [15]:
x_axis = 'K'
y_axis = 'minor_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_mu, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


# III. Analysis : mu

Here are the **plot parameters** to be played with :

In [16]:
ERROR_BARS = True #Wether to plot the error bars
NORMALIZE = False

To analyse the impact of mu, we the **value of K is fixed**:

In [17]:
df_fix_K = df[df['K']==4] # Currently K in [2,4,8,12]


## a. best_acc_te

In [18]:
x_axis = 'mu'
y_axis = 'best_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, normalize=NORMALIZE)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## b. worst_acc_te

In [19]:
x_axis = 'K'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
plot_graph_all(df_fix_mu, x_axis, y_axis,error_bars=ERROR_BARS, normalize=NORMALIZE)

count                                        sem            \
method           dro  erm   jtt  rwg  rwy  subg  suby       dro       erm   
K name                                                                      
2 AWA            5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.092018  0.132032   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.153852  0.075890   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.148002  0.181808   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.024087  0.030690   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.028583  0.014470   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.035873  0.026982   
  plt-net        5.0  5.0  10.0  5.0  5.0  10.0  10.0  0.057312  0.026696   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.126936  0.029560   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.139816  0.055556   
4 AWA            5.0  5.0   5.0  5.0  5.0  10.0   5.0  0.108243  0.077551   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.221646  0.163497   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.106451  0.113567   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.139484  0.075740   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.114954  0.089838   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.071216  0.080840   
  plt-net        2.0  2.0   NaN  NaN  NaN   NaN   NaN  0.500000  0.500000   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.064142  0.172577   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.174136  0.028571   
8 AWA            5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.102515  0.054578   
  airplanes      5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.037579  0.024922   
  birds          5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.092179  0.102183   
  dogs           5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.048009  0.037613   
  medical-leaf   5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.075835  0.073532   
  plt-doc        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.063775  0.095028   
  plt-net        5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.047504  0.033082   
  resisc         5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.146942  0.107846   
  texture-dtd    5.0  5.0   5.0  5.0  5.0   5.0   5.0  0.000000  0.000000   

                                                                  
method               jtt       rwg       rwy      subg      suby  
K name                                                            
2 AWA           0.127909  0.109882  0.094693  0.000000  0.126117  
  airplanes     0.013539  0.105813  0.091734  0.080541  0.000000  
  birds         0.132811  0.194167  0.178965  0.147394  0.140133  
  dogs          0.025268  0.049911  0.027393  0.000000  0.016629  
  medical-leaf  0.034319  0.030321  0.036364  0.000000  0.104876  
  plt-doc       0.051883  0.078977  0.035866  0.000000  0.007857  
  plt-net       0.032402  0.058839  0.112816  0.020325  0.022271  
  resisc        0.043077  0.142499  0.112168  0.178500  0.097840  
  texture-dtd   0.088696  0.113709  0.079679  0.112279  0.036679  
4 AWA           0.086022  0.117733  0.065932  0.021874  0.069323  
  airplanes     0.094765  0.077495  0.088507  0.095966  0.089001  
  birds         0.144277  0.029769  0.039364  0.152823  0.114685  
  dogs          0.054108  0.124285  0.094144  0.018526  0.030303  
  medical-leaf  0.040334  0.100090  0.151309  0.058469  0.061261  
  plt-doc       0.154957  0.122786  0.075370  0.060465  0.037117  
  plt-net            NaN       NaN       NaN       NaN       NaN  
  resisc        0.000000  0.055215  0.144363  0.125319  0.109844  
  texture-dtd   0.094993  0.143372  0.091897  0.085714  0.093953  
8 AWA           0.059629  0.087441  0.109612  0.000000  0.010777  
  airplanes     0.039818  0.088315  0.022476  0.000000  0.000000  
  birds         0.006226  0.024975  0.110346  0.132418  0.118754  
  dogs          0.044613  0.085468  0.048524  0.020000  0.045569  
  medical-leaf  0.000000  0.1

In [20]:
x_axis = 'mu'
y_axis = 'worst_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


## c. relative_acc

In [21]:
x_axis = 'mu'
y_axis = 'relative_grp_acc_te' #'mean_grp_acc_te'
for dataset_name in DATASETS.values():
    plot_graph(df_fix_K, x_axis, y_axis, dataset_name,error_bars=ERROR_BARS, methods=None)

dogs


medical-leaf


texture-dtd


birds


AWA


plt-net


resisc


plt-doc


airplanes


ALL


# IV. Examples 

## a. Dataset difficulty :

In [22]:
def plot_violin(df, metric, category):
    """
    Creates and displays a violin plot for the given DataFrame.
    Parameters:
    - df: pandas DataFrame containing the data to plot.
    - metric: str, the name of the column in df containing the values to plot.
    - category: str, the name of the column in df representing different categories to separate the violins.

    """
    fig = go.Figure()
    categories = df[category].unique()
    for cat in categories:
        cat_data = df[df[category] == cat][metric]
        fig.add_trace(go.Violin(y=cat_data, name=cat, box_visible=True, meanline_visible=True))

    fig.update_layout(title=f"{metric} by {category} for dataset",
                      yaxis_title=metric,
                      legend_title=category)

    # Show the figure
    fig.show()

In [23]:
plot_violin(df_fix_mu,'best_acc_te','method')

In [24]:
plot_violin(df_fix_mu_K2,'worst_grp_acc_te','method')

In [25]:
plot_violin(df_fix_mu_K12,'worst_grp_acc_te','method')

In [26]:
plot_violin(df_fix_mu,'worst_grp_acc_te','method')

In [27]:
plot_violin(df_fix_mu,'best_acc_te','name')

In [53]:
plot_violin(df_fix_mu_K12,'best_acc_te','name')

In [29]:
plot_violin(df_fix_mu,'worst_grp_acc_te','name')

In [30]:
plot_violin(df_fix_mu_K12,'best_acc_te','name')

In [54]:
df_1 = df_fix_mu[df_fix_mu['name'] == 'dogs']
df_1 = df_1[df_1['K'] == 12]
df_1 = df_1[df_1['method'] == 'erm']
df_1['best_acc_te']

2844    0.827342
2847    0.820479
2848    0.813617
2849    0.828649
2850    0.828105
Name: best_acc_te, dtype: float64

In [56]:
df_2 = df_fix_mu[df_fix_mu['name'] == 'medical-leaf']
df_2 = df_2[df_2['K'] == 12]
df_2 = df_2[df_2['method'] == 'erm']
df_2['best_acc_te']

2516    0.910802
2517    0.883642
2518    0.916049
2519    0.899691
2520    0.904630
Name: best_acc_te, dtype: float64

In [33]:
df_2['best_acc_te'].std()/(5)**.5 # We check the computation from previous error bar in graphs

0.013566206179819724